In [66]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Yola_fit').getOrCreate()



In [67]:
df = spark.read.csv('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Pricing_fallback_rule.csv', header='True')



In [68]:
df.createOrReplaceTempView('Fallback_rule')

In [69]:
df.show()

+------------------+----------+--------------------+--------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+---+-----------------+---------------+
|         item_code|      date|item_gtin_comparable|  item_main_category| item_sub_category_1|item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin|nov|shop_gross_margin|shop_pc2_margin|
+------------------+----------+--------------------+--------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+---+-----------------+---------------+
|000000001000052666|2020-02-16|          comparab

In [70]:
import pandas as pd
import numpy as np
import patsy as pat



In [71]:
pdf_sap = pd.read_excel('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Lieferanten_Master_Data_SAP_V2.xlsx',header=3, usecols=['SAP_ID','Supplier_Name','Type'])#,'Type'])
pdf_sap.dropna(inplace=True)
print(pdf_sap.head(100))


#df_sap = spark.createDataFrame(pdf_sap)



       SAP_ID                         Supplier_Name         Type
2    100000.0                    Art of Living GmbH  Marketplace
3    100001.0            Actona Company AS (Inland)     Domestic
4    100002.0   Actona Company A/S (Hauptwerk SHANG   FOB Import
5    100003.0                      Actual Diffusion  Marketplace
6    100004.0                   Arte M GmbH & Co.KG     Domestic
..        ...                                   ...          ...
103  100109.0   Duo Collection Import Vertriebsgese     Domestic
104  100110.0                          David & Luna     Domestic
105  100111.0  design3000 Vertriebsgesellschaft mbH  Marketplace
106  100112.0             Dreisitz-design Sp.z.o.o.     Domestic
107  100113.0                          Dan-Form A/S     Domestic

[100 rows x 3 columns]


In [72]:
print(pdf_sap.dtypes)
pdf_sap_filt = pdf_sap.astype({'SAP_ID': 'int32', 'Supplier_Name': 'string','Type':'string'})
print(pdf_sap_filt.dtypes)



SAP_ID           float64
Supplier_Name     object
Type              object
dtype: object
SAP_ID            int32
Supplier_Name    string
Type             string
dtype: object


In [73]:
df_sap = spark.createDataFrame(pdf_sap_filt)
df_sap.createOrReplaceTempView('SAP_id')

In [74]:
df_sap_INJOIN = spark.sql("""SELECT fbr.*, sap.Type
                                from Fallback_rule as fbr
                              INNER JOIN SAP_id as sap on sap.SAP_ID = CAST(fbr.item_supplier_regular_id as INT) """)

In [75]:
df_sap_INJOIN.createOrReplaceTempView('IN_JOIN_Table')

In [76]:
df_sap_INJOIN.show()

+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+
|000000001000097652|20

In [77]:
df_fin_table = spark.sql(""" SELECT ijt.*, shop_pc2_margin/nov  as shop_pc2_margin_nov_ratio,
        CASE
           when ijt.item_inco_terms = 'FOB'
               then 'Import'
           when LOWER(ijt.Type) LIKE 'eastern europe'
               then 'EE'
           when LOWER(ijt.Type) LIKE 'import'
               then 'Import'
           else 'Domestic' END                                                              as sourcing_location
            from IN_JOIN_Table as ijt
""")
#shop_pc2_margin/NOV

In [78]:
df_fin_table.show()
df_fin_table.createOrReplaceTempView('ijt_table')


+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+-------------------------+-----------------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|shop_pc2_margin_nov_ratio|sourcing_location|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-

In [79]:
df_fin_table_v2 = spark.sql(""" SELECT *,
                            CASE
                            WHEN inventory_flag in ('former','laso','tto','riskshs') then 'wh'
                            WHEN inventory_flag in ('warehouse-undefined','mto','keb','realshsmov') then 'mto'
                            ELSE 'dropship' end as wh_flag_new
                            FROM ijt_table
                            WHERE inventory_flag not in ('former','warehouse-undefined','UNKNOWN')""")

In [80]:
df_fin_table_v2.show()
df_fin_table_v2.createOrReplaceTempView('Fin_table')

+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+-------------------------+-----------------+-----------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|shop_pc2_margin_nov_ratio|sourcing_location|wh_flag_new|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+-------------------

In [82]:
df_exclusivity_clean  = spark.sql("""SELECT DISTINCT exclusivity_clean  from Fin_table """)
df_item_main_category = spark.sql("""SELECT DISTINCT item_main_category from Fin_table """)
df_sourcing_location  = spark.sql("""SELECT DISTINCT sourcing_location  from Fin_table """)
df_wh_flag  = spark.sql("""SELECT DISTINCT wh_flag_new  from Fin_table """)


In [83]:
#df_test.show()
exclusivity_clean_list = df_exclusivity_clean.rdd.map(lambda x: x.exclusivity_clean).collect()
item_main_category_list = df_item_main_category.rdd.map(lambda x: x.item_main_category).collect()
sourcing_location_list = df_sourcing_location.rdd.map(lambda x: x.sourcing_location).collect()
wh_flag_list = df_wh_flag.rdd.map(lambda x: x.wh_flag_new).collect()

In [84]:
print(exclusivity_clean_list,'\n',item_main_category_list,'\n',sourcing_location_list,'\n',wh_flag_list)


['exclusive', 'other', 'thirdParty', 'ownDeveloped', 'whiteLabel'] 
 ['TEPPICHE & BODENBELÄGE', 'HAUSHALT', 'ESSZIMMERTISCHE', 'LEUCHTEN', 'POLSTER', 'BÜRO', 'KINDER', 'FLURMÖBEL', 'SCHLAFZIMMERMÖBEL', 'WOHNEN', 'GARTEN', 'KÜCHE', 'BOUTIQUE', 'BABY', 'HEIMTEXTILIEN', 'MATRATZEN&LATTENROSTE', 'BETTEN', 'BAD', 'ESSZIMMERSTÜHLE&BÄNKE'] 
 ['Import', 'EE', 'Domestic'] 
 ['mto', 'wh', 'dropship']


In [90]:
pdf_fin = df_fin_table_v2.toPandas()


In [91]:
print(pdf_fin.head())


            item_code        date item_gtin_comparable item_main_category  \
0  000000001000097652  2020-06-09           comparable             WOHNEN   
1  000000001000097578  2020-05-14           comparable             WOHNEN   
2  000000001000097652  2020-05-16           comparable             WOHNEN   
3  000000001000097495  2020-05-31           comparable             WOHNEN   
4  000000001000097495  2020-06-06           comparable             WOHNEN   

     item_sub_category_1 item_sub_category_2 item_supplier_regular_id  \
0                 REGALE          WANDREGALE                   100565   
1  KOMMODEN & SIDEBOARDS            KOMMODEN                   100565   
2                 REGALE          WANDREGALE                   100565   
3  KOMMODEN & SIDEBOARDS          SIDEBOARDS                   100565   
4  KOMMODEN & SIDEBOARDS          SIDEBOARDS                   100565   

  item_inco_terms exclusivity_clean item_supplier_regular  ...  \
0             DDP        thirdPa

In [94]:
#MODEL IMPLEMENTATION
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

list_of_num_columns = ['wh_flag','shop_pc2_margin','pocket_pc2_margin','nov','shop_gross_margin']
for col in list_of_num_columns:
    pdf_fin[col]=pd.to_numeric(pdf_fin[col])
#pdf_fin = pdf_fin.astype({'wh_flag':'int32', 'shop_pc2_margin':'int32','item_main_category':'string','exclusivity_clean':'string','sourcing_location':'string' })


In [95]:
print(pdf_fin.dtypes)


item_code                     object
date                          object
item_gtin_comparable          object
item_main_category            object
item_sub_category_1           object
item_sub_category_2           object
item_supplier_regular_id      object
item_inco_terms               object
exclusivity_clean             object
item_supplier_regular         object
inventory_flag                object
wh_flag                        int64
item_final_price_rule         object
sellability_laso_de           object
pocket_pc2_margin              int64
nov                            int64
shop_gross_margin              int64
shop_pc2_margin                int64
Type                          object
shop_pc2_margin_nov_ratio    float64
sourcing_location             object
wh_flag_new                   object
dtype: object


In [96]:
WH_with_fe_false = smf.ols("shop_pc2_margin_nov_ratio ~  C(wh_flag_new) + C(sourcing_location) + C(exclusivity_clean) +C(item_main_category)  - 1 ",  pdf_fin,hasconst = False).fit()
print(WH_with_fe_false.summary())
#

                                    OLS Regression Results                                    
Dep. Variable:     shop_pc2_margin_nov_ratio   R-squared (uncentered):                   0.416
Model:                                   OLS   Adj. R-squared (uncentered):              0.416
Method:                        Least Squares   F-statistic:                          1.879e+04
Date:                       Tue, 02 Feb 2021   Prob (F-statistic):                        0.00
Time:                               18:19:28   Log-Likelihood:                     -3.8108e+05
No. Observations:                     712887   AIC:                                  7.622e+05
Df Residuals:                         712860   BIC:                                  7.625e+05
Df Model:                                 27                                                  
Covariance Type:                   nonrobust                                                  
                                                  

In [97]:
parameter_map = zip(list(WH_with_fe_false.params),list(WH_with_fe_false.bse))
print(list(parameter_map))


[(0.19988380475201836, 0.008431921572507505), (0.2041726094409428, 0.008373760030907253), (0.2110754246377614, 0.008253274472447362), (0.027901031096840452, 0.0021241028679631554), (0.09381553867982155, 0.0015601802077108025), (-0.015032267508521158, 0.0026721141972766873), (0.000740712957794154, 0.002559427778974828), (-0.027381968962465614, 0.002112594626776865), (-0.021632394918299963, 0.001527111838332581), (0.1600288543413531, 0.008523574234324462), (0.07005252684357215, 0.008352038069708655), (0.040344771980309564, 0.008673318534883383), (0.1449014596688934, 0.008442489055596192), (0.15505299793375912, 0.008270617522631338), (0.13478296034503645, 0.008519327165740385), (0.14855596198875543, 0.008376026719740649), (0.025515957239480386, 0.008945360566979359), (-0.1169196678081528, 0.04661336054243398), (0.06383464577567079, 0.008593869391903968), (0.02292286127459361, 0.009950126827463776), (0.16138667803174905, 0.015306602932888724), (0.10143749407988883, 0.00826416927620698), (0

In [98]:
params = pd.Series(WH_with_fe_false.params, name = 'beta')
errors  = pd.Series(WH_with_fe_false.bse, name = 'beta_err')

result_df = pd.concat([params,errors],axis=1)
print(result_df)
#for name, value in ndarray:
#    print(name,value)


                                                     beta  beta_err
C(wh_flag_new)[dropship]                         0.199884  0.008432
C(wh_flag_new)[mto]                              0.204173  0.008374
C(wh_flag_new)[wh]                               0.211075  0.008253
C(sourcing_location)[T.EE]                       0.027901  0.002124
C(sourcing_location)[T.Import]                   0.093816  0.001560
C(exclusivity_clean)[T.other]                   -0.015032  0.002672
C(exclusivity_clean)[T.ownDeveloped]             0.000741  0.002559
C(exclusivity_clean)[T.thirdParty]              -0.027382  0.002113
C(exclusivity_clean)[T.whiteLabel]              -0.021632  0.001527
C(item_main_category)[T.BAD]                     0.160029  0.008524
C(item_main_category)[T.BETTEN]                  0.070053  0.008352
C(item_main_category)[T.BOUTIQUE]                0.040345  0.008673
C(item_main_category)[T.BÜRO]                    0.144901  0.008442
C(item_main_category)[T.ESSZIMMERSTÜHLE&BÄNKE]  

In [99]:
def map_of_parameters(wh_flag_new ='mto', sourcing_location = 'EE', exclusivity_clean = 'other', item_main_category ='GARTEN' ):
    map_of_parameter = {'wh_flag_new': wh_flag_new,
                        'sourcing_location' : 'T.'+sourcing_location,
                        'exclusivity_clean' : 'T.'+exclusivity_clean,
                        'item_main_category': 'T.'+item_main_category}
    if sourcing_location == 'Domestic':
        del map_of_parameter['sourcing_location']
    if exclusivity_clean == 'exclusive':
        del map_of_parameter['exclusivity_clean']
    if item_main_category == 'BABY':
        del map_of_parameter['item_main_category']

    #print(map_of_parameter)
    return map_of_parameter


In [119]:

fin_coeff = pd.DataFrame()

def calc_beta(df =result_df, map = { 'wh_flag_new':'mto', 'sourcing_location' : 'T.EE', 'exclusivity_clean' : 'T.other', 'item_main_category' :'T.GARTEN'}):
    prefix = 'C('
    suffix = ']'

    for key in map.keys():
        map[key]= prefix+ key + ')[' + map[key] + suffix
    #print(map)
    df_filt = pd.DataFrame()
    #print(list(map.values()))
    df_filt = df.loc[list(map.values())]
    #print(df_filt)


    print('Final coefficient = ', df_filt['beta'].sum().round(3), '+/-', np.sqrt((df_filt['beta_err']**2).sum()).round(3), '\n')
    return (df_filt['beta'].sum().round(3), np.sqrt((df_filt['beta_err']**2).sum()).round(3))




In [120]:
results = calc_beta(result_df,map_of_parameters('mto','Domestic','exclusive','BABY'))
print(results)

Final coefficient =  0.204 +/- 0.008 

(0.204, 0.008)


In [121]:
#Loop over all categories
#f = open('beta_results.txt','w')
map_results = {'item_main_category':[], 'exclusivity_clean':[], 'sourcing_location':[],'wh_flag_new':[], 'beta':[],'beta_err':[]}
for imc in item_main_category_list:
    for ec in exclusivity_clean_list:
        for sl in sourcing_location_list:
            for wh in wh_flag_list:

                results = calc_beta(result_df,map_of_parameters(wh,sl,ec,imc))
                map_results['item_main_category'].append(imc)
                map_results['exclusivity_clean'].append(ec)
                map_results['sourcing_location'].append(sl)
                map_results['wh_flag_new'].append(wh)
                map_results['beta'].append(results[0])
                map_results['beta_err'].append(results[1])
print(map_results)


Final coefficient =  0.43 +/- 0.012 

Final coefficient =  0.437 +/- 0.012 

Final coefficient =  0.426 +/- 0.012 

Final coefficient =  0.364 +/- 0.012 

Final coefficient =  0.371 +/- 0.012 

Final coefficient =  0.36 +/- 0.012 

Final coefficient =  0.336 +/- 0.012 

Final coefficient =  0.343 +/- 0.012 

Final coefficient =  0.332 +/- 0.012 

Final coefficient =  0.415 +/- 0.012 

Final coefficient =  0.422 +/- 0.012 

Final coefficient =  0.411 +/- 0.012 

Final coefficient =  0.349 +/- 0.012 

Final coefficient =  0.356 +/- 0.012 

Final coefficient =  0.345 +/- 0.012 

Final coefficient =  0.321 +/- 0.012 

Final coefficient =  0.328 +/- 0.012 

Final coefficient =  0.317 +/- 0.012 

Final coefficient =  0.403 +/- 0.012 

Final coefficient =  0.409 +/- 0.012 

Final coefficient =  0.398 +/- 0.012 

Final coefficient =  0.337 +/- 0.012 

Final coefficient =  0.344 +/- 0.012 

Final coefficient =  0.332 +/- 0.012 

Final coefficient =  0.309 +/- 0.012 

Final coefficient =  0.316 

In [115]:
df_fit_result = pd.DataFrame(map_results)

print(df_fit_result.head(1000))

         item_main_category exclusivity_clean sourcing_location wh_flag_new  \
0    TEPPICHE & BODENBELÄGE         exclusive            Import         mto   
1    TEPPICHE & BODENBELÄGE         exclusive            Import          wh   
2    TEPPICHE & BODENBELÄGE         exclusive            Import    dropship   
3    TEPPICHE & BODENBELÄGE         exclusive                EE         mto   
4    TEPPICHE & BODENBELÄGE         exclusive                EE          wh   
..                      ...               ...               ...         ...   
850   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE          wh   
851   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE    dropship   
852   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic         mto   
853   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic          wh   
854   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic    dropship   

      beta  beta_err  
0    0.430     0.012  
1    

In [122]:
df_fit_result.to_csv('./fit_result_v0_new_wh_flag.csv')



In [124]:
WH_with_inter = smf.ols("shop_pc2_margin_nov_ratio ~  C(wh_flag_new) + C(sourcing_location) + C(exclusivity_clean) +C(item_main_category) +"
                           " C(item_main_category)*C(wh_flag_new) - 1"#C(item_main_category)*C(sourcing_location) +"
                           #"C(item_main_category)*C(exclusivity_clean) - 1 ",
                            ,pdf_fin,hasconst = False).fit()
print(WH_with_inter.summary())



                                    OLS Regression Results                                    
Dep. Variable:     shop_pc2_margin_nov_ratio   R-squared (uncentered):                   0.416
Model:                                   OLS   Adj. R-squared (uncentered):              0.416
Method:                        Least Squares   F-statistic:                              8336.
Date:                       Tue, 02 Feb 2021   Prob (F-statistic):                        0.00
Time:                               18:26:30   Log-Likelihood:                     -3.8072e+05
No. Observations:                     712887   AIC:                                  7.616e+05
Df Residuals:                         712826   BIC:                                  7.623e+05
Df Model:                                 61                                                  
Covariance Type:                   nonrobust                                                  
                                                  

In [125]:
parameter_map = zip(list(WH_with_inter.params),list(WH_with_inter.bse))
print(list(parameter_map))


[(0.09388141298255785, 0.01971478655809988), (0.23353728558741765, 0.0208952022447253), (0.23546041752153835, 0.00991200887109848), (0.021486377348804916, 0.002178204845631977), (0.08847509927123559, 0.001642404611730939), (-0.016420225321842008, 0.0027039972805415194), (0.00021864095573739806, 0.002584111121189844), (-0.030969876055953437, 0.0021579501629703656), (-0.024490117146006407, 0.001551505947903756), (0.258355846820403, 0.019943525866723773), (0.2591880212387227, 0.06273359210828912), (0.15346662361534255, 0.01987103743671596), (0.27193672205369285, 0.020601502284395782), (0.2426247020338672, 0.020108648368075993), (0.26675493229750846, 0.022821390739339788), (0.24502544069493787, 0.020078518716952532), (0.16039016132734912, 0.020191071569926524), (-0.00753670538858673, 0.049883604308101855), (0.18298504657252374, 0.0198369163425887), (0.12647348951985926, 0.022483399748333518), (0.3074317174974752, 0.029031289418033575), (0.20212389125135918, 0.019692484694787366), (0.216411

In [126]:
params = pd.Series(WH_with_inter.params, name = 'beta_one')
errors  = pd.Series(WH_with_inter.bse, name = 'beta_one_err')

result_df = pd.concat([params,errors],axis=1)
print(result_df)
#for name, value in ndarray:
#    print(name,value)


                                                    beta_one  beta_one_err
C(wh_flag_new)[dropship]                            0.093881      0.019715
C(wh_flag_new)[mto]                                 0.233537      0.020895
C(wh_flag_new)[wh]                                  0.235460      0.009912
C(sourcing_location)[T.EE]                          0.021486      0.002178
C(sourcing_location)[T.Import]                      0.088475      0.001642
...                                                      ...           ...
C(item_main_category)[T.MATRATZEN&LATTENROSTE]:... -0.106083      0.026318
C(item_main_category)[T.POLSTER]:C(wh_flag_new)... -0.153224      0.028476
C(item_main_category)[T.SCHLAFZIMMERMÖBEL]:C(wh... -0.165502      0.031820
C(item_main_category)[T.TEPPICHE & BODENBELÄGE]... -0.174425      0.022563
C(item_main_category)[T.WOHNEN]:C(wh_flag_new)[... -0.141296      0.022311

[63 rows x 2 columns]


In [130]:
def map_of_parameters(wh_flag_new ='mto', sourcing_location = 'EE', exclusivity_clean = 'other', item_main_category ='GARTEN'):
    map_of_parameter = {'wh_flag_new': wh_flag_new,
                        'sourcing_location'         : 'T.'+ sourcing_location,
                        'exclusivity_clean'         : 'T.'+ exclusivity_clean,
                        'item_main_category'        : 'T.'+ item_main_category,
                        'item_main_category_wh_flag': 'T.'+ item_main_category + ':' + wh_flag_new  }
    if sourcing_location == 'Domestic':
        del map_of_parameter['sourcing_location']
    if exclusivity_clean == 'exclusive':
        del map_of_parameter['exclusivity_clean']
    if item_main_category == 'BABY':
        del map_of_parameter['item_main_category']
        del map_of_parameter['item_main_category_wh_flag']
    if wh_flag_new == 'dropship' and item_main_category != 'BABY':
        del map_of_parameter['item_main_category_wh_flag']

    print(map_of_parameter)
    return map_of_parameter




In [134]:
fin_coeff = pd.DataFrame()

def calc_beta(df =result_df, map = { 'wh_flag_new':'mto', 'sourcing_location' : 'T.EE', 'exclusivity_clean' : 'T.other', 'item_main_category' :'T.GARTEN', 'item_main_category_wh_flag': 'T.GARTEN:mto'}):
    prefix = 'C('
    suffix = ']'
    wh_flag_temp= map['wh_flag_new']
    for key in map.keys():
        map[key]= prefix+ key + ')[' + map[key] + suffix
        if key == 'item_main_category_wh_flag' :
            map[key] = map['item_main_category'] + ':C(wh_flag_new)[T.' + wh_flag_temp + suffix
    print(map)
    df_filt = pd.DataFrame()
    print(list(map.values()))
    df_filt = df.loc[list(map.values())]
    print(df_filt)


    print('Final coefficient = ', df_filt['beta_one'].sum().round(3), '+/-', np.sqrt((df_filt['beta_one_err']**2).sum()).round(3), '\n')
    return (df_filt['beta_one'].sum().round(3), np.sqrt((df_filt['beta_one_err']**2).sum()).round(3))

In [139]:
results = calc_beta(result_df,map_of_parameters('mto','Import','ownDeveloped','BAD'))
print(results)

{'wh_flag_new': 'mto', 'sourcing_location': 'T.Import', 'exclusivity_clean': 'T.ownDeveloped', 'item_main_category': 'T.BAD', 'item_main_category_wh_flag': 'T.BAD:mto'}
{'wh_flag_new': 'C(wh_flag_new)[mto]', 'sourcing_location': 'C(sourcing_location)[T.Import]', 'exclusivity_clean': 'C(exclusivity_clean)[T.ownDeveloped]', 'item_main_category': 'C(item_main_category)[T.BAD]', 'item_main_category_wh_flag': 'C(item_main_category)[T.BAD]:C(wh_flag_new)[T.mto]'}
['C(wh_flag_new)[mto]', 'C(sourcing_location)[T.Import]', 'C(exclusivity_clean)[T.ownDeveloped]', 'C(item_main_category)[T.BAD]', 'C(item_main_category)[T.BAD]:C(wh_flag_new)[T.mto]']
                                                    beta_one  beta_one_err
C(wh_flag_new)[mto]                                 0.233537      0.020895
C(sourcing_location)[T.Import]                      0.088475      0.001642
C(exclusivity_clean)[T.ownDeveloped]                0.000219      0.002584
C(item_main_category)[T.BAD]                        0.

In [140]:
#Loop over all categories
#f = open('beta_results.txt','w')
map_results = {'item_main_category':[], 'exclusivity_clean':[], 'sourcing_location':[],'wh_flag_new':[], 'beta_one':[],'beta_one_err':[]}
for imc in item_main_category_list:
    for ec in exclusivity_clean_list:
        for sl in sourcing_location_list:
            for wh in wh_flag_list:

                results = calc_beta(result_df,map_of_parameters(wh,sl,ec,imc))
                map_results['item_main_category'].append(imc)
                map_results['exclusivity_clean'].append(ec)
                map_results['sourcing_location'].append(sl)
                map_results['wh_flag_new'].append(wh)
                map_results['beta_one'].append(results[0])
                map_results['beta_one_err'].append(results[1])
print(map_results)


{'wh_flag_new': 'mto', 'sourcing_location': 'T.Import', 'item_main_category': 'T.TEPPICHE & BODENBELÄGE', 'item_main_category_wh_flag': 'T.TEPPICHE & BODENBELÄGE:mto'}
{'wh_flag_new': 'C(wh_flag_new)[mto]', 'sourcing_location': 'C(sourcing_location)[T.Import]', 'item_main_category': 'C(item_main_category)[T.TEPPICHE & BODENBELÄGE]', 'item_main_category_wh_flag': 'C(item_main_category)[T.TEPPICHE & BODENBELÄGE]:C(wh_flag_new)[T.mto]'}
['C(wh_flag_new)[mto]', 'C(sourcing_location)[T.Import]', 'C(item_main_category)[T.TEPPICHE & BODENBELÄGE]', 'C(item_main_category)[T.TEPPICHE & BODENBELÄGE]:C(wh_flag_new)[T.mto]']
                                                    beta_one  beta_one_err
C(wh_flag_new)[mto]                                 0.233537      0.020895
C(sourcing_location)[T.Import]                      0.088475      0.001642
C(item_main_category)[T.TEPPICHE & BODENBELÄGE]     0.263320      0.019958
C(item_main_category)[T.TEPPICHE & BODENBELÄGE]... -0.239366      0.293310
Final

In [144]:
df_fit_inter_result = pd.DataFrame(map_results)

print(df_fit_inter_result.head(1000))

         item_main_category exclusivity_clean sourcing_location wh_flag_new  \
0    TEPPICHE & BODENBELÄGE         exclusive            Import         mto   
1    TEPPICHE & BODENBELÄGE         exclusive            Import          wh   
2    TEPPICHE & BODENBELÄGE         exclusive            Import    dropship   
3    TEPPICHE & BODENBELÄGE         exclusive                EE         mto   
4    TEPPICHE & BODENBELÄGE         exclusive                EE          wh   
..                      ...               ...               ...         ...   
850   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE          wh   
851   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE    dropship   
852   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic         mto   
853   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic          wh   
854   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic    dropship   

     beta_one  beta_one_err  
0       0.346        

In [145]:
#Let's join the two dataframe
print(df_fit_result.head(1000))

         item_main_category exclusivity_clean sourcing_location wh_flag_new  \
0    TEPPICHE & BODENBELÄGE         exclusive            Import         mto   
1    TEPPICHE & BODENBELÄGE         exclusive            Import          wh   
2    TEPPICHE & BODENBELÄGE         exclusive            Import    dropship   
3    TEPPICHE & BODENBELÄGE         exclusive                EE         mto   
4    TEPPICHE & BODENBELÄGE         exclusive                EE          wh   
..                      ...               ...               ...         ...   
850   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE          wh   
851   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE    dropship   
852   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic         mto   
853   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic          wh   
854   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic    dropship   

      beta  beta_err  
0    0.430     0.012  
1    

In [146]:
final_df = pd.concat([df_fit_result,df_fit_inter_result[['beta_one','beta_one_err']]], axis = 1, join='inner')
print(final_df.head(1000))


         item_main_category exclusivity_clean sourcing_location wh_flag_new  \
0    TEPPICHE & BODENBELÄGE         exclusive            Import         mto   
1    TEPPICHE & BODENBELÄGE         exclusive            Import          wh   
2    TEPPICHE & BODENBELÄGE         exclusive            Import    dropship   
3    TEPPICHE & BODENBELÄGE         exclusive                EE         mto   
4    TEPPICHE & BODENBELÄGE         exclusive                EE          wh   
..                      ...               ...               ...         ...   
850   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE          wh   
851   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel                EE    dropship   
852   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic         mto   
853   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic          wh   
854   ESSZIMMERSTÜHLE&BÄNKE        whiteLabel          Domestic    dropship   

      beta  beta_err  beta_one  beta_one_err  
0   

In [148]:
final_df.to_csv('./fit_result_v1_new_wh_flag.csv')